<a href="https://colab.research.google.com/github/imirenechen/AI-study-club/blob/main/01_unet_cars_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Move to your current working directory
%cd drive/MyDrive/Colab\ Notebooks/Cars

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Cars


In [2]:
# Import all the packages
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

In [3]:
# Build one of the main components - DoubleConv - for UNet
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
    )
  def forward(self, x):
    return self.conv(x)

In [4]:
# Build UNet from scrach
class UNet(nn.Module):
  def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
    super().__init__()
    self.downs = nn.ModuleList()
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature
    self.bottlenect = DoubleConv(features[-1], features[-1]*2)
    self.ups = nn.ModuleList()
    for feature in reversed(features):
      self.ups.append(nn.ConvTranspose2d(feature*2, feature, 2, 2))
      self.ups.append(DoubleConv(feature*2, feature))
    self.final_conv = nn.Conv2d(features[0], out_channels, 1)

  def forward(self, x):
    skip_connections = []
    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = F.max_pool2d(x, (2, 2))
    x = self.bottlenect(x)
    skip_connections.reverse() # inplace
    for i in range(0, len(self.ups), 2):
      x = self.ups[i](x)
      skip_connection = skip_connections[i//2]
      concat = torch.cat((skip_connection, x), dim=1) # N x C x H x W
      x = self.ups[i+1](concat)
    return self.final_conv(x)

In [5]:
# Test model only
model = UNet()
toy_data = torch.ones([16, 3, 240, 160])
out = model(toy_data)
print(out.shape)

torch.Size([16, 1, 240, 160])


In [6]:
# Create an UNet model object
model = UNet()

# Move the model to GPU
model = model.cuda()

In [7]:
# Build CustomDataset for loading data from Google Drive
class CustomDataset(Dataset):
  def __init__(self, image_dir, mask_dir, transform):
    super().__init__()
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform
    self.images = os.listdir(self.image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    image_path = os.path.join(self.image_dir, self.images[i])
    mask_path = os.path.join(self.mask_dir, self.images[i].replace(".jpg", "_mask.gif"))
    image = np.array(Image.open(image_path))
    mask = np.array(Image.open(mask_path).convert("L"))
    return self.transform(image), self.transform(mask)

In [8]:
# Check the device we are using is GPU or CPU
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cuda


In [9]:
# Constants for UNet model training process
BATCH_SIZE = 16
NUM_EPOCHS = 3
IMG_WIDTH = 240
IMG_HEIGHT = 160

In [10]:
# Load data
all_data = CustomDataset('small_train', 'small_train_masks', T.Compose([T.ToTensor(), T.Resize((IMG_HEIGHT, IMG_WIDTH))]))

In [11]:
# Split data into train and val
train_data, val_data = torch.utils.data.random_split(all_data, [0.7, 0.3])

In [12]:
# Create loader for mini-batch gradient descent
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
# The loss function for bianry classification
loss_function = nn.BCELoss()

# Choosing Adam as our optimizer
optimizer = optim.Adam(model.parameters())

In [14]:
def train(model, num_epochs, train_loader, optimizer, print_every=30):
  for epoch in range(num_epochs):
    for count, (x, y) in enumerate(train_loader):
      model.train()
      x = x.to(device)
      y = y.to(device)
      out = model(x)
      if count % print_every == 0:
        eval(model, val_loader, epoch)
      out = torch.sigmoid(out)
      loss = loss_function(out, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [15]:
def eval(model, val_loader, epoch):
  model.eval()
  num_correct = 0
  num_pixels = 0
  with torch.no_grad():
    for x, y in val_loader:
      x = x.to(device)
      y = y.to(device)
      out_img = model(x)
      probability = torch.sigmoid(out_img)
      predictions = probability>0.5
      num_correct += (predictions==y).sum()
      num_pixels += BATCH_SIZE*IMG_WIDTH*IMG_HEIGHT
  print(f'Epoch[{epoch+1}] Acc: {num_correct/num_pixels}')


In [16]:
train(model, NUM_EPOCHS, train_loader, optimizer)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch[1] Acc: 0.790369987487793
Epoch[1] Acc: 0.8479331731796265
Epoch[1] Acc: 0.9744017124176025
Epoch[2] Acc: 0.9809433817863464
Epoch[2] Acc: 0.9876761436462402
Epoch[2] Acc: 0.9887269139289856
Epoch[3] Acc: 0.9901547431945801
Epoch[3] Acc: 0.9917691946029663
Epoch[3] Acc: 0.9908760786056519
